In [ ]:
#import skimage.io

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import scipy.signal

In [ ]:
from util import plotstyle

In [ ]:
plotstyle.load('screen_dark')
#plotstyle.load('pretty_print')
#plotstyle.load('print')

In [ ]:
from util import bfp

# Calibrate using a Calibration Dataset
## Create Wrapper Object

In [ ]:
calibration_dataset = bfp.SliceDataset()

## Load Image Data

In [ ]:
calibration_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211111/calibration_test_01.tif") )

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( calibration_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('x [px]')
ax.set_ylabel('y [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

plt.tight_layout()
plt.show()

## Generate Mask

In [ ]:
calibration_dataset.gen_mask()

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( calibration_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('x [px]')
ax.set_ylabel('y [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

## Generate Regions of Interest from the Mask

In [ ]:
calibration_dataset.gen_rois()

In [ ]:
[ roi.to_string() for roi in calibration_dataset.rois ]

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=calibration_dataset.rois[i].to_extent()
    )
    if i==0:
        ax.set_ylabel('y [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('x [px]')
    ax.set_title("ROI {i}".format(i=i))

#plt.tight_layout()
plt.show()

## Compute the Calibration Constant

In [ ]:
calibration = calibration_dataset.get_calibration()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=calibration_dataset.rois[i].to_extent()
    )
    ax.fill_betweenx( 
        #np.arange(calibration_dataset.roi_mids.shape[1]),
        calibration_dataset.rois[i].Y(),
        #np.convolve( calibration_dataset.roi_mids[i]-calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        #np.convolve( calibration_dataset.roi_mids[i]+calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        calibration_dataset.roi_mids[i] - calibration_dataset.roi_devs[i] + calibration_dataset.rois[i].xmin, 
        calibration_dataset.roi_mids[i] + calibration_dataset.roi_devs[i] + calibration_dataset.rois[i].xmin, 
        color='r', alpha=2*plotstyle.err_alpha() )
    #ax.plot( 
    #    calibration_dataset.roi_mids[i] + calibration_dataset.rois[i].xmin, 
    #    np.arange(calibration_dataset.roi_mids.shape[1]),
    #    color='r', lw=1 )
    ax.plot( 
        [ np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin, np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin], 
        [ -0.5, calibration_dataset.roi_mids.shape[1]-0.5 ],
        color=plotstyle.monochrome_fg(), lw=1, ls=':' )
    
    limits = calibration_dataset.rois[i].to_extent()
    ax.set_xlim( xmin=limits[0], xmax=limits[1] )
    ax.set_ylim( ymin=limits[2], ymax=limits[3] )
    del limits
    
    if i==0:
        ax.set_ylabel('y [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('x [px]')
    ax.set_title("ROI {i}".format(i=i))

#plt.tight_layout()
plt.show()

In [ ]:
calibration.to_dict()

# Run Analysis on a Slice Image

In [ ]:
slice_dataset = bfp.SliceDataset()
slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211112/scatterer01/lineprofile-12500µm.tif") )


In [ ]:
slice_dataset.gen_mask()


In [ ]:
fig = plt.figure(figsize=(11,11), dpi=100)

axs = fig.add_gridspec(2, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( slice_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('x [px]')
ax.set_ylabel('y [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

ax = fig.add_subplot(axs[ 1 , 0 ])

im = ax.imshow( slice_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('x [px]')
ax.set_ylabel('y [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

In [ ]:
slice_dataset.gen_rois()

In [ ]:
slice_dataset.map_rois()

In [ ]:
slice_dataset.correct_top_bottom_bg()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int( slice_dataset.SLICE.width() + slice_dataset.SMEAR.width() ) + 10 )

ax = fig.add_subplot( axs[ 0 , :int( slice_dataset.SLICE.width() ) ] )
im = ax.imshow( 
    slice_dataset.SLICE(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=slice_dataset.SLICE.to_extent()
)
ax.set_ylabel('y [px]')
ax.set_xlabel('x [px]')
ax.set_title("SLICE")

ax.grid()



EXTENT = slice_dataset.SMEAR.to_extent()
EXTENT[0] -= slice_dataset.x0_px
EXTENT[1] -= slice_dataset.x0_px
EXTENT[0] *= calibration.px_to_lda()
EXTENT[1] *= calibration.px_to_lda()

ax = fig.add_subplot( axs[ 0 , int( slice_dataset.SLICE.width() )+10: ] )
im = ax.imshow( 
    slice_dataset.SMEAR(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=EXTENT
)
ax.set(yticklabels=[])
ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
ax.set_title("SMEAR")

ax.grid()
    

    
#plt.tight_layout()
plt.show()